# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
from cassandra.cluster import Cluster
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Get absolute filepath of event_data directory
filepath = os.getcwd() + '/event_data'

# join the file path and roots with the subdirectories using glob
file_path_list = glob.glob(os.path.join(filepath,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Create an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

    # Read csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        
        # Create a csv reader object 
        csvreader = csv.reader(csvfile) 
        
        # Skip header row (column names)
        next(csvreader)
        
        # Extract each row of data and append        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print('Full combined CSV data contains', len(full_data_rows_list), 'rows.')           

Full combined CSV data contains 8056 rows.


In [4]:
# Create a smaller event data csv file called event_datafile_full.csv that will be inserted 
# into the Apache Cassandra tables

# Create custom dialect to parse CSV
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# Create new csv to store data and instantiate a writer object
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    
    # Name columns
    writer.writerow([
        'artist',
        'first_name',
        'gender',
        'item_in_session',
        'last_name',
        'length',    
        'level',
        'location',
        'session_id',
        'song',
        'user_id'
    ])
    
    # Iterate through each row in the data list
    for row in full_data_rows_list:
        
        # Filter out rows from CSVs that don't deal with song plays
        if row[0] != '':
            
            # Store only relevant features
            writer.writerow((
                row[0], 
                row[2], 
                row[3], 
                row[4], 
                row[5], 
                row[6], 
                row[7], 
                row[8], 
                row[12], 
                row[13], 
                row[16]
            ))

# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('Filtered CSV File contains', sum(1 for line in f), 'rows.')

Filtered CSV File contains 6821 rows.


# Part II. Complete the Apache Cassandra coding portion of your project. 

Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance in the Cassandra container 
try:
    # Create a connection to the docker image
    cluster = Cluster(['127.0.0.1'], port=9042)
    session = cluster.connect()

except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS 
          sparkify 
        WITH REPLICATION = {
            'class': 'SimpleStrategy', 
            'replication_factor': 1
        }
    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1

**Query:** Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4.

The primary key is (`session_id`, `item_in_session`) because we put those would go in our `WHERE` clause. I have set the partition key to `session_id` because we would want to store all of the information for a single session on the same node, rather than to have it scattered across nodes. `item_in_session` is the clustering column because we might want to order by this column or filter by it.

### CQL Queries and Table Creation

In [8]:
# CQL Queries: Give me the artist, song title and song's length in the 
# music app history that was heard during  sessionId = 338, and itemInSession  = 4
select_query_1 = """
    SELECT 
      artist, 
      song, 
      length 
    FROM 
      song_info 
    WHERE 
      session_id = 338 AND 
      item_in_session = 4
"""

create_query_1 = """
    CREATE TABLE IF NOT EXISTS
      song_info (
        session_id int,
        item_in_session int,
        artist text,
        length float,
        song text,
        PRIMARY KEY (session_id, item_in_session)
      )
"""

insert_query_1 = """
    INSERT INTO 
      song_info (
        session_id,
        item_in_session,
        artist,
        length,
        song
      )
    VALUES
      (%s, %s, %s, %s, %s)
"""
        
try:
    session.execute(create_query_1)
except Exception as e:
    print(e)                    

### Data Insertion

In [9]:
# Read filtered CSV
with open('event_datafile_new.csv', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    
    # Skip header row (column names)
    next(csvreader)
    
    for line in csvreader:
        
        # Insert each row into table
        session.execute(
            insert_query_1, 
            (int(line[8]), int(line[3]), line[0], float(line[5]), line[9])
        )

### Verify Data Insertion

`pandas_factory()` and the two subsequent lines of code set up the infrastructure for all other queries to be displayed as DataFrames.

In [10]:
# Define function that will instantiate a dataframe row factory
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

# Specify to use the pandas_factory
session.row_factory = pandas_factory

# Default is 50000, but our tables are pretty small so ignore
session.default_fetch_size = None

# Execute query 
rows = session.execute(select_query_1)
df = rows._current_rows
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Query 2

**Query:** Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.

The primary key is (`user_id`, `session_id`, `item_in_session`) because they will appear in our WHERE clause for a query like this. `user_id` is the partition key because we will want to store all information by the same user on a single node. Next, because we know we want to filter by `session_id` and return results sorted by `item_in_session`, then we must use those two columns as clustering columns in that order.

### CQL Queries and Table Creation

In [11]:
# PRIMARY KEY (user_id, session_id, item_in_session)
select_query_2 = """
    SELECT 
      artist, 
      song, 
      first_name, 
      last_name 
    FROM 
      users_songs 
    WHERE 
      user_id = 10 AND 
      session_id = 182
"""

create_query_2 = """
    CREATE TABLE IF NOT EXISTS
      users_songs (
        user_id int,
        session_id int,
        item_in_session int,
        artist text,
        first_name text,
        last_name text,
        song text,
        PRIMARY KEY (user_id, session_id, item_in_session)
      )
"""

insert_query_2 = """
    INSERT INTO 
      users_songs (
        user_id,
        session_id,
        item_in_session,
        artist,
        first_name,
        last_name,
        song
      )
    VALUES
      (%s, %s, %s, %s, %s, %s, %s)
"""
        
try:
    session.execute(create_query_2)
except Exception as e:
    print(e)                      

### Data Insertion

In [12]:
# Read filtered CSV
with open('event_datafile_new.csv', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    
    # Skip header row (column names)
    next(csvreader)
    
    for line in csvreader:
        
        # Insert each row into table
        session.execute(
            insert_query_2, 
            (
                int(line[10]),
                int(line[8]),
                int(line[3]), 
                line[0], 
                line[1], 
                line[4], 
                line[9]   
            )
        )

### Verify Data Insertion

In [13]:
# Execute query 
rows = session.execute(select_query_2)
df = rows._current_rows
df

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Query 3

**Query:** Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The primary key is (`song`, `user_id`) with partition key `song` because we want to keep all users who listen to the same song on a single node and filter by song. We use `user_id` as a clustering column because we can optionally filter by that column.


### CQL Queries and Table Creation

In [14]:
select_query_3 = """
    SELECT 
      first_name, 
      last_name 
    FROM 
      user_name 
    WHERE 
      song = 'All Hands Against His Own'
"""

create_query_3 = """
    CREATE TABLE IF NOT EXISTS
      user_name (
        song text,
        user_id int,
        last_name text,
        first_name text,
        PRIMARY KEY (song, user_id)
      )
"""

insert_query_3 = """
    INSERT INTO 
      user_name (
        song,
        user_id,
        first_name,
        last_name
      )
    VALUES
      (%s, %s, %s, %s)
"""
        
try:
    session.execute(create_query_3)
except Exception as e:
    print(e)

### Data Insertion

In [15]:
# Read filtered CSV
with open('event_datafile_new.csv', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    
    # Skip header row (column names)
    next(csvreader)
    
    for line in csvreader:
        
        # Insert each row into table
        session.execute(
            insert_query_3, (line[9], int(line[10]), line[1], line[4])
        )

### Verify Data Insertion

In [16]:
# Execute query 
rows = session.execute(select_query_3)
df = rows._current_rows
df

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [17]:
drop_query_1 = "DROP TABLE song_info"
drop_query_2 = "DROP TABLE users_songs"
drop_query_3 = "DROP TABLE user_name"

drop_queries = [drop_query_1, drop_query_2, drop_query_3]

for query in drop_queries:
    session.execute(query)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()